In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


In [2]:
# Pulled dataset from HF, dropped unneeded index column
df = pd.read_csv("hf://datasets/maharshipandya/spotify-tracks-dataset/dataset.csv").drop(
    "Unnamed: 0", axis=1
)
df["duration_s"] = df["duration_ms"] / 1000
df_nodupe = df.drop_duplicates(subset=["track_id"])